In [1]:
from transformers import Trainer, TrainingArguments
from transformers import AutoTokenizer, AutoModelForSequenceClassification,  AutoModelForMaskedLM
import pandas as pd
from datasets import Dataset, load_dataset, ClassLabel
import numpy as np
from typing import List
from transformers import DataCollatorWithPadding
import evaluate
import torch
import random
import os

In [2]:
!nvidia-smi

Tue Jul 25 19:40:44 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.161.03   Driver Version: 470.161.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A6000    On   | 00000000:4F:00.0 Off |                    0 |
| 30%   28C    P8    22W / 300W |      3MiB / 45631MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA RTX A6000    On   | 00000000:52:00.0 Off |                    0 |
| 30%   

In [3]:
%env CUDA_VISIBLE_DEVICES=2
%env TOKENIZERS_PARALLELISM=false
#torch.manual_seed(0)
#torch.use_deterministic_algorithms(True)
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"

env: CUDA_VISIBLE_DEVICES=2
env: TOKENIZERS_PARALLELISM=false


In [4]:
id2label ={0: "formal", 1: "informal"}
label2id = {"formal" : 0, "informal" : 1}

In [5]:
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")
#model = AutoModelForSequenceClassification.from_pretrained("xlm-roberta-base", num_labels=2, id2label=id2label, label2id=label2id)
model = AutoModelForMaskedLM.from_pretrained("xlm-roberta-base", num_labels=2, id2label=id2label, label2id=label2id)
#model = AutoModelForMaskedLM.from_pretrained("xlm-roberta-base", num_labels=2)

#model = AutoModelForMaskedLM.from_pretrained("distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id)


In [5]:
# Annotate Data and Shuffle
hon_data_path = "/home/sumire/thesis/LLM_Contextual_Prompt_MT/data/contrastive-controlled-mt/IWSLT2022/data/"

train1_fp_f = hon_data_path + "train/en-ja/formality-control.train.telephony.en-ja.formal.ja"
train1_fp_if = hon_data_path + "train/en-ja/formality-control.train.telephony.en-ja.informal.ja"
train2_fp_f = hon_data_path + "train/en-ja/formality-control.train.topical-chat.en-ja.formal.ja"
train2_fp_if = hon_data_path + "train/en-ja/formality-control.train.topical-chat.en-ja.informal.ja"

test_fp_f = hon_data_path + "test/en-ja/formality-control.test.en-ja.formal.ja"
test_fp_if = hon_data_path + "test/en-ja/formality-control.test.en-ja.informal.ja"

train_with_label = []
test_with_label = []

for fp in [train1_fp_f, train1_fp_if, train2_fp_f, train2_fp_if, test_fp_f, test_fp_if]: 
    dataset = load_dataset("text", data_files=fp)
    
    if fp == train1_fp_f or fp == train2_fp_f:
        for sent in dataset["train"]["text"]:
            sent = sent + ", 0" 
            train_with_label.append(sent)
    
    elif fp == train1_fp_if or fp == train2_fp_if:
        for sent in dataset["train"]["text"]:
            sent = sent + ", 1"
            train_with_label.append(sent)
            
    elif fp == test_fp_f :
        for sent in dataset["train"]["text"]:
            sent = sent + ", 0"
            test_with_label.append(sent)
    
    elif fp == test_fp_if :
        for sent in dataset["train"]["text"]:
            sent = sent + ", 1"
            test_with_label.append(sent)

random.shuffle(train_with_label)
random.shuffle(test_with_label)

with open (hon_data_path + "modified/train_en-ja.ja", "w") as train_file:
    for item in train_with_label:
        # write each item on a new line
        train_file.write("%s\n" % item)
        
with open (hon_data_path + "modified/test_en-ja.ja", "w") as test_file:
    for item in test_with_label:
        # write each item on a new line
        test_file.write("%s\n" % item)
print('Done')


Using custom data configuration default-aa4180dfc0d8544e
Found cached dataset text (/home/sumire/.cache/huggingface/datasets/text/default-aa4180dfc0d8544e/0.0.0/99cc88223027054f94ce0c7fd69d10eb172910fa0615671283a3c8e5e7af2f9c)


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-8d8cb62fa76f7d67
Found cached dataset text (/home/sumire/.cache/huggingface/datasets/text/default-8d8cb62fa76f7d67/0.0.0/99cc88223027054f94ce0c7fd69d10eb172910fa0615671283a3c8e5e7af2f9c)


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-c5e57c6f711b908f
Found cached dataset text (/home/sumire/.cache/huggingface/datasets/text/default-c5e57c6f711b908f/0.0.0/99cc88223027054f94ce0c7fd69d10eb172910fa0615671283a3c8e5e7af2f9c)


...

Using custom data configuration default-4a02738e5d40d937
Found cached dataset text (/home/sumire/.cache/huggingface/datasets/text/default-4a02738e5d40d937/0.0.0/99cc88223027054f94ce0c7fd69d10eb172910fa0615671283a3c8e5e7af2f9c)


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-483622c8cf9f8077
Found cached dataset text (/home/sumire/.cache/huggingface/datasets/text/default-483622c8cf9f8077/0.0.0/99cc88223027054f94ce0c7fd69d10eb172910fa0615671283a3c8e5e7af2f9c)


  0%|          | 0/1 [00:00<?, ?it/s]

Done


In [6]:
print (len(test_with_label))

NameError: name 'test_with_label' is not defined

In [7]:
tgt_lang = "ja"
hon_data_path = "/home/sumire/thesis/LLM_Contextual_Prompt_MT/data/contrastive-controlled-mt/IWSLT2022/data/"

data_files = { "train": hon_data_path+"modified/train_en-ja.ja", "test": hon_data_path+"modified/test_en-ja.ja"}
dataset = load_dataset("text", data_files=data_files)
dataset

Using custom data configuration default-0a2611c093b6474c
Found cached dataset text (/home/sumire/.cache/huggingface/datasets/text/default-0a2611c093b6474c/0.0.0/99cc88223027054f94ce0c7fd69d10eb172910fa0615671283a3c8e5e7af2f9c)


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text'],
        num_rows: 1188
    })
})

In [8]:
inputs =  [sent.split(", ")[0] for sent in dataset["test"]["text"]]

In [9]:
# Add labels
train_labels = [sent.split(", ")[1] for sent in dataset["train"]["text"]]
test_labels = [sent.split(", ")[1] for sent in dataset["test"]["text"]]

dataset["train"]=dataset["train"].add_column("labels", train_labels)
dataset["test"]=dataset["test"].add_column("labels", test_labels)

In [10]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'labels'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'labels'],
        num_rows: 1188
    })
})

In [11]:
#dataset = dataset.cast_column('labels', ClassLabel(num_classes=2, names=["Formal", "Informal"]))
dataset = dataset.class_encode_column('labels')
dataset["train"].features

Loading cached processed dataset at /home/sumire/.cache/huggingface/datasets/text/default-0a2611c093b6474c/0.0.0/99cc88223027054f94ce0c7fd69d10eb172910fa0615671283a3c8e5e7af2f9c/cache-53c785a19aaad065.arrow
Loading cached processed dataset at /home/sumire/.cache/huggingface/datasets/text/default-0a2611c093b6474c/0.0.0/99cc88223027054f94ce0c7fd69d10eb172910fa0615671283a3c8e5e7af2f9c/cache-b49de0a535926365.arrow


{'text': Value(dtype='string', id=None),
 'labels': ClassLabel(names=['0', '1'], id=None)}

In [12]:
dataset["train"][0]["text"].split(", ")[0]

'私は買い物の9割をそこでしていますが、ある種類の紙や特定のレシピのために探してもない商品もある。'

In [13]:
def preprocess_function(data):
    inputs = [sent.split(", ")[0] for sent in data["text"]]
    #inputs = [kshot + sent + ' = ' for doc in data["doc"] for sent in doc["en"] ][:50]
    return tokenizer(inputs, truncation=True, padding=True, return_tensors="pt")

In [14]:
tokenized_dataset = dataset.map(preprocess_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Loading cached processed dataset at /home/sumire/.cache/huggingface/datasets/text/default-0a2611c093b6474c/0.0.0/99cc88223027054f94ce0c7fd69d10eb172910fa0615671283a3c8e5e7af2f9c/cache-5f2e070f77bc0c75.arrow


  0%|          | 0/2 [00:00<?, ?ba/s]

In [15]:
def compute_metrics(eval_preds):
    
    preds, labels = eval_preds
    #eval_preds[1] = [label for label in data["labels"]]
    #preds = eval_preds[1]
    accuracy = evaluate.load("accuracy")
    f1 = evaluate.load("f1")
    
    preds = np.argmax(preds, axis=1)
    result = {}
    result["accuracy"] = accuracy.compute(predictions=preds, references=labels)
    result["f1"] = f1.compute(predictions=preds, references=labels)
    print (result)
    return result

In [16]:
training_args = TrainingArguments(
    output_dir="./results/hon_xlm-r",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

The following columns in the training set don't have a corresponding argument in `XLMRobertaForMaskedLM.forward` and have been ignored: text. If text are not expected by `XLMRobertaForMaskedLM.forward`,  you can safely ignore this message.
/home/sumire/miniconda3/envs/llm_mt/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 2000
  Num Epochs = 2
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 250
  Number of trainable parameters = 278295186
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than

ValueError: Expected input batch_size (1440) to match target batch_size (16).

In [22]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis", model="xlm-roberta-base")
classifier(inputs)

loading configuration file config.json from cache at /home/sumire/.cache/huggingface/hub/models--xlm-roberta-base/snapshots/77de1f7a7e5e737aead1cd880979d4f1b3af6668/config.json
Model config XLMRobertaConfig {
  "_name_or_path": "xlm-roberta-base",
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.25.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 250002
}

loading configuration file config.json from cache at /home/sumire/.cache/huggingface/hub/models--xlm-roberta-base

[{'label': 'LABEL_0', 'score': 0.5117709040641785},
 {'label': 'LABEL_0', 'score': 0.5132048726081848},
 {'label': 'LABEL_0', 'score': 0.5131912231445312},
 {'label': 'LABEL_0', 'score': 0.503860354423523},
 {'label': 'LABEL_0', 'score': 0.5126904249191284},
 {'label': 'LABEL_0', 'score': 0.5159482359886169},
 {'label': 'LABEL_0', 'score': 0.5020590424537659},
 {'label': 'LABEL_0', 'score': 0.5043507218360901},
 {'label': 'LABEL_0', 'score': 0.5159978270530701},
 {'label': 'LABEL_0', 'score': 0.5094629526138306},
 {'label': 'LABEL_0', 'score': 0.5092950463294983},
 {'label': 'LABEL_0', 'score': 0.5163185000419617},
 {'label': 'LABEL_0', 'score': 0.5007461309432983},
 {'label': 'LABEL_0', 'score': 0.510414183139801},
 {'label': 'LABEL_0', 'score': 0.5141968131065369},
 {'label': 'LABEL_0', 'score': 0.5049339532852173},
 {'label': 'LABEL_0', 'score': 0.5119562149047852},
 {'label': 'LABEL_0', 'score': 0.5134588479995728},
 {'label': 'LABEL_0', 'score': 0.5119021534919739},
 {'label': 'LA

# Huggingface Text Classification Tutorial

In [30]:
"""
from datasets import load_dataset

imdb = load_dataset("imdb")
"""
hon_data_path = "/home/sumire/thesis/LLM_Contextual_Prompt_MT/data/contrastive-controlled-mt/IWSLT2022/data/"

data_files = { "train": hon_data_path+"modified/train_en-ja.ja", "test": hon_data_path+"modified/test_en-ja.ja"}
dataset = load_dataset("text", data_files=data_files)

# Add labels
train_labels = [sent.split(", ")[1] for sent in dataset["train"]["text"]]
test_labels = [sent.split(", ")[1] for sent in dataset["test"]["text"]]

dataset["train"]=dataset["train"].add_column("labels", train_labels)
dataset["test"]=dataset["test"].add_column("labels", test_labels)

#dataset = dataset.cast_column('labels', ClassLabel(num_classes=2, names=["Formal", "Informal"]))
dataset = dataset.class_encode_column('labels')
dataset["train"].features

Using custom data configuration default-0a2611c093b6474c
Found cached dataset text (/home/sumire/.cache/huggingface/datasets/text/default-0a2611c093b6474c/0.0.0/99cc88223027054f94ce0c7fd69d10eb172910fa0615671283a3c8e5e7af2f9c)


  0%|          | 0/2 [00:00<?, ?it/s]

Loading cached processed dataset at /home/sumire/.cache/huggingface/datasets/text/default-0a2611c093b6474c/0.0.0/99cc88223027054f94ce0c7fd69d10eb172910fa0615671283a3c8e5e7af2f9c/cache-53c785a19aaad065.arrow
Loading cached processed dataset at /home/sumire/.cache/huggingface/datasets/text/default-0a2611c093b6474c/0.0.0/99cc88223027054f94ce0c7fd69d10eb172910fa0615671283a3c8e5e7af2f9c/cache-b49de0a535926365.arrow


{'text': Value(dtype='string', id=None),
 'labels': ClassLabel(names=['0', '1'], id=None)}

In [32]:
#imdb["test"][0]
dataset["test"][0]

{'text': '私のメールアドレスはその時には無効になっていて使えないので、伝えたくないですが。, 0', 'labels': 0}

In [33]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

loading configuration file config.json from cache at /home/sumire/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.25.1",
  "vocab_size": 30522
}

loading file vocab.txt from cache at /home/sumire/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/vocab.txt
loading file tokenizer.json from cache at /home/sumire/.cache/huggingface/hub/models--distilber

In [34]:
"""
def preprocess_function(examples):
    inputs = examples["text"][:10]
    return tokenizer(examples["text"], truncation=True)
"""
def preprocess_function(data):
    inputs = [sent.split(", ")[0] for sent in data["text"]]
    #inputs = [kshot + sent + ' = ' for doc in data["doc"] for sent in doc["en"] ][:50]
    return tokenizer(inputs, truncation=True, padding=True, return_tensors="pt")

In [35]:
#tokenized_imdb = imdb.map(preprocess_function, batched=True)
tokenized_dataset = dataset.map(preprocess_function, batched=True)


Loading cached processed dataset at /home/sumire/.cache/huggingface/datasets/text/default-0a2611c093b6474c/0.0.0/99cc88223027054f94ce0c7fd69d10eb172910fa0615671283a3c8e5e7af2f9c/cache-e3fc394d26affe85.arrow
Loading cached processed dataset at /home/sumire/.cache/huggingface/datasets/text/default-0a2611c093b6474c/0.0.0/99cc88223027054f94ce0c7fd69d10eb172910fa0615671283a3c8e5e7af2f9c/cache-27bc438f2a4703e9.arrow


In [36]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [37]:
import evaluate

accuracy = evaluate.load("accuracy")

In [38]:
import numpy as np

"""
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    print (predictions, labels)
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)
"""
def compute_metrics(eval_preds):
    
    preds, labels = eval_preds
    #eval_preds[1] = [label for label in data["labels"]]
    #preds = eval_preds[1]
    accuracy = evaluate.load("accuracy")
    f1 = evaluate.load("f1")
    
    preds = np.argmax(preds, axis=1)
    result = {}
    result["accuracy"] = accuracy.compute(predictions=preds, references=labels)
    result["f1"] = f1.compute(predictions=preds, references=labels)
    print (result)
    return result

In [39]:
id2label = {0: "formal", 1: "informal"}
label2id = {"formal": 0, "informal": 1}

In [40]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id)
#model = AutoModelForMaskedLM.from_pretrained("xlm-roberta-base", num_labels=2, id2label=id2label, label2id=label2id)

#tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

loading configuration file config.json from cache at /home/sumire/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "formal",
    "1": "informal"
  },
  "initializer_range": 0.02,
  "label2id": {
    "formal": 0,
    "informal": 1
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.25.1",
  "vocab_size": 30522
}

loading weights file pytorch_model.bin from cache at /home/sumire/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda

In [41]:
training_args = TrainingArguments(
    output_dir="./hon_distribert",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 2000
  Num Epochs = 2
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 250
  Number of trainable parameters = 66955010
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a m

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.167770,{'accuracy': 0.9570707070707071},{'f1': 0.9573221757322177}
2,No log,0.168956,{'accuracy': 0.9587542087542088},{'f1': 0.958927074601844}


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1188
  Batch size = 16
Trainer is attempting to log a value of "{'accuracy': 0.9570707070707071}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'f1': 0.9573221757322177}" of type <class 'dict'> for key "eval/f1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


{'accuracy': {'accuracy': 0.9570707070707071}, 'f1': {'f1': 0.9573221757322177}}


Saving model checkpoint to my_awesome_model/checkpoint-125
Configuration saved in my_awesome_model/checkpoint-125/config.json
Model weights saved in my_awesome_model/checkpoint-125/pytorch_model.bin
tokenizer config file saved in my_awesome_model/checkpoint-125/tokenizer_config.json
Special tokens file saved in my_awesome_model/checkpoint-125/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1188
  Batch size = 16
Trainer is attempting to log a value of "{'accuracy': 0.9587542087542088}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'f1': 0.958927074

{'accuracy': {'accuracy': 0.9587542087542088}, 'f1': {'f1': 0.958927074601844}}


Saving model checkpoint to my_awesome_model/checkpoint-250
Configuration saved in my_awesome_model/checkpoint-250/config.json
Model weights saved in my_awesome_model/checkpoint-250/pytorch_model.bin
tokenizer config file saved in my_awesome_model/checkpoint-250/tokenizer_config.json
Special tokens file saved in my_awesome_model/checkpoint-250/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from my_awesome_model/checkpoint-125 (score: 0.16776955127716064).


TrainOutput(global_step=250, training_loss=0.2471763153076172, metrics={'train_runtime': 26.7555, 'train_samples_per_second': 149.502, 'train_steps_per_second': 9.344, 'total_flos': 147990921936000.0, 'train_loss': 0.2471763153076172, 'epoch': 2.0})

In [57]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1188
  Batch size = 16


Trainer is attempting to log a value of "{'accuracy': 0.9688552188552189}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'f1': 0.9687763713080169}" of type <class 'dict'> for key "eval/f1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


{'accuracy': {'accuracy': 0.9688552188552189}, 'f1': {'f1': 0.9687763713080169}}


{'eval_loss': 0.10766708105802536,
 'eval_accuracy': {'accuracy': 0.9688552188552189},
 'eval_f1': {'f1': 0.9687763713080169},
 'eval_runtime': 2.7435,
 'eval_samples_per_second': 433.021,
 'eval_steps_per_second': 27.337,
 'epoch': 2.0}

### Findings
1. "distilbert-base-uncased" won't give the problem that ValueError: Expected input batch_size (1440) to match target batch_size (16), but "xlm-base" does
2. remove_column _traom ?? gives the problem that eval_preds does not hand over the labels,